# Implementing range slider

A range slider differs from a normal slider in that it returns a range of values between two defined points, in this case between two points in time.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from jupyter_dash import JupyterDash
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
from dash.dependencies import Input, Output

df1 = pd.read_csv('block_1.csv')
df1[['day','time']] = df1['tstp'].str.split(' ', expand = True)
df1.drop(columns = 'tstp', inplace = True)

df1['energy(kWh/hh)'] = pd.to_numeric(df1['energy(kWh/hh)'], errors='coerce')
df1[['year','month','date']] = df1.day.str.split('-', expand = True)

In [2]:
df1

,LCLid,energy(kWh/hh),day,time,year,month,date
0,MAC000323,0.488,2012-03-06,14:00:00.0000000,2012,03,06
1,MAC000323,0.449,2012-03-06,14:30:00.0000000,2012,03,06
2,MAC000323,0.424,2012-03-06,15:00:00.0000000,2012,03,06
3,MAC000323,0.439,2012-03-06,15:30:00.0000000,2012,03,06
4,MAC000323,0.291,2012-03-06,16:00:00.0000000,2012,03,06
...,...,...,...,...,...,...,...
1515860,MAC004529,0.213,2014-02-27,22:00:00.0000000,2014,02,27
1515861,MAC004529,0.227,2014-02-27,22:30:00.0000000,2014,02,27
1515862,MAC004529,0.229,2014-02-27,23:00:00.0000000,2014,02,27
1515863,MAC004529,0.220,2014-02-27,23:30:00.0000000,2014,02,27


In [3]:
filt = df1['year'] == '2013'
df_2013 = df1[filt]

df_2013['month'] = df_2013['month'].astype('int64')
df_2013['date'] = df_2013['date'].astype('int64')
df_2013.dtypes

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


LCLid              object
energy(kWh/hh)    float64
day                object
time               object
year               object
month               int64
date                int64
dtype: object

In [4]:
grouped_2013 = df_2013.groupby(['day','month','LCLid'])
grouped_2013_df = grouped_2013['energy(kWh/hh)'].mean().reset_index()
grouped_2013_df

,day,month,LCLid,energy(kWh/hh)
0,2013-01-01,1,MAC000323,0.502208
1,2013-01-01,1,MAC000379,0.894292
2,2013-01-01,1,MAC000386,0.286500
3,2013-01-01,1,MAC000535,0.780042
4,2013-01-01,1,MAC000557,1.543000
...,...,...,...,...
17803,2013-12-31,12,MAC003166,0.662521
17804,2013-12-31,12,MAC003182,0.146646
17805,2013-12-31,12,MAC003212,0.311417
17806,2013-12-31,12,MAC003840,0.397063


In [5]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    dcc.Graph(id='graph-with-slider'),
    dcc.RangeSlider(
        id='month-slider',
        min=grouped_2013_df['month'].min(),
        max=grouped_2013_df['month'].max(),
        marks={
            1: 'Jan',
            2: 'Feb',
            3: 'Mar',
            4: 'Apr',
            5: 'May',
            6: 'Jun',
            7: 'Jul',
            8: 'Aug',
            9: 'Sep',
            10: 'Oct',
            11: 'Nov',
            12: 'Dec'
        },
        value=[grouped_2013_df['month'].min(),grouped_2013_df['month'].max()],
        step=None
    )

])


@app.callback(
    Output('graph-with-slider', 'figure'),
    [Input('month-slider', 'value')])
def update_figure(selected_month):
    # selected_month is value as a list
    # requires boolean vector
    filtered_df = grouped_2013_df[(grouped_2013_df['month'] >= selected_month[0]) & (grouped_2013_df['month'] <= selected_month[1])]

    fig = px.scatter(filtered_df, x="day", y="energy(kWh/hh)", 
                     color="LCLid", hover_name="LCLid")

    

    return fig

app.run_server(mode = 'inline')